In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/projectvalues/Project_Data/labels-test.tsv
/kaggle/input/projectvalues/Project_Data/arguments-validation.tsv
/kaggle/input/projectvalues/Project_Data/arguments-training.tsv
/kaggle/input/projectvalues/Project_Data/README.md
/kaggle/input/projectvalues/Project_Data/arguments-test.tsv
/kaggle/input/projectvalues/Project_Data/labels-validation.tsv
/kaggle/input/projectvalues/Project_Data/labels-training.tsv
/kaggle/input/value-categories/value-categories (1).json


In [2]:
import pandas as pd
import json

# Load the arguments data (assuming it has columns 'id', 'premise')
arguments_df = pd.read_csv('/kaggle/input/projectvalues/Project_Data/arguments-training.tsv', delimiter='\t')

# Load the value labels data (assuming it has columns 'id', 'value_label')
labels_df = pd.read_csv('/kaggle/input/projectvalues/Project_Data/labels-training.tsv', delimiter='\t')

# Load the value descriptions from a JSON file
with open('/kaggle/input/value-categories/value-categories (1).json', 'r') as file:
    value_descriptions = json.load(file)
    
labels_long_df = labels_df.melt(id_vars='Argument ID', var_name='value_category', value_name='label')
labels_long_df = labels_long_df[labels_long_df['value_category'] != 'Universalism: objectivity']
labels_long_df['value_description'] = labels_long_df['value_category'].apply(lambda x: value_descriptions[x.lower().replace(": ","-")]['personal-motivation'])
combined_df = pd.merge(arguments_df, labels_long_df, left_on='Argument ID', right_on='Argument ID')
combined_df['Argument'] = combined_df.apply(
    lambda row: f"{row['Stance']} {row['Conclusion']} by saying {row['Premise']}",
    axis=1
)

In [3]:
final_df=combined_df
df_majority = final_df[final_df.label == 0]
df_minority = final_df[final_df.label == 1]

# Determine the number of instances you want to keep from the majority class
# For example, you might want to have a 1:1 ratio
number_of_instances = len(df_minority)

# Downsample the majority class
df_majority_downsampled = df_majority.sample(n=number_of_instances)

# Combine the downsampled majority class with the minority class to get a balanced dataset
balanced_df = pd.concat([df_majority_downsampled, df_minority])

# Shuffle the dataset to mix the two classes well
balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

In [4]:
balanced_df

,Argument ID,Conclusion,Stance,Premise,value_category,label,value_description,Argument
0,A19165,We should ban telemarketing,against,telemarketing is a perfectly valid marketing a...,Self-direction: action,1,It is important to make own decisions about li...,against We should ban telemarketing by saying ...
1,A01006,We should end the use of economic sanctions,against,sometimes economic sanctions are the only thin...,Security: societal,1,Country should protect itself against all thre...,against We should end the use of economic sanc...
2,A21166,We should end the use of economic sanctions,in favor of,we could not support economic sanctions as it ...,Security: societal,1,Country should protect itself against all thre...,in favor of We should end the use of economic ...
3,A30066,We should fight for the abolition of nuclear w...,against,nuclear weapons should be retained as they act...,Self-direction: action,0,It is important to make own decisions about li...,against We should fight for the abolition of n...
4,A22123,We should ban telemarketing,in favor of,telemarketing intrudes on the personal lives o...,Security: personal,1,"Avoid dangerous situations, value personal sec...",in favor of We should ban telemarketing by say...
...,...,...,...,...,...,...,...,...
34627,A18099,We should abolish intellectual property rights,in favor of,maintaining intellectual property rights for a...,Power: resources,1,"Having lots of money for the power it brings, ...",in favor of We should abolish intellectual pro...
34628,A30263,We should abolish the right to keep and bear arms,against,the freedom to bear arms has always been a bas...,Universalism: concern,0,"Protecting the weak and vulnerable, care about...",against We should abolish the right to keep an...
34629,A27491,We should legalize sex selection,in favor of,people can have strong feelings about which se...,Hedonism,0,"Having a good time, enjoying life’s pleasures ...",in favor of We should legalize sex selection b...
34630,D01096,We should subsidize higher education institutions,in favor of,Indian students perform brilliantly when they ...,Tradition,0,"Maintain traditional beliefs and values, follo...",in favor of We should subsidize higher educati...


In [5]:
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.nn.functional import cross_entropy
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from sklearn.metrics import accuracy_score,f1_score
model_name = 'pepa/roberta-base-snli'
config = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                             return_dict=True,
                                                             output_hidden_states=False,
                                                             hidden_dropout_prob=0.3,  # Set dropout probability for hidden layers
                                                             attention_probs_dropout_prob=0.3)  # Set dropout probability for attention layers

model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)
inputs = tokenizer(list(balanced_df['Argument']), list(balanced_df['value_description']), padding=True, truncation=True, return_tensors="pt")
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
labels = torch.tensor(balanced_df['label'].values)

config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
train_dataset = TensorDataset(input_ids, attention_mask, labels)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
optimizer = AdamW(model.parameters(), lr=2e-5)
epochs=3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
import numpy as np
# Define a training loop
for epoch in range(epochs):
    model.train()
    total_loss = 0
    predictions = []
    true_labels = []
    steps=1
    i=0
    for batch in train_dataloader:
        b_input_ids, b_attention_mask, b_labels = batch
        b_input_ids = b_input_ids.to(device)
        b_attention_mask = b_attention_mask.to(device)
        b_labels = b_labels.to(device)
        
        # Clear any previously calculated gradients
        optimizer.zero_grad()
        
        # Perform a forward pass. This will return logits.
        outputs = model(b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        
        # Calculate loss using the outputs and the labels
        loss = outputs[0]
        total_loss += loss.item()
        logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend(np.argmax(logits, axis=1).flatten())
        true_labels.extend(label_ids.flatten())
        
        # Perform a backward pass to calculate gradients
        loss.backward()
        
        # Update parameters and take a step using the computed gradient
        optimizer.step()
        
        # Update the learning rate
        scheduler.step()
        if steps % 200 == 0:
            interim_f1 = f1_score(true_labels, predictions, average='macro')
            print(f"Epoch {epoch}, Step {steps}, Loss: {total_loss / steps:.4f}, Interim F1 Score: {interim_f1:.4f}")
            predictions = []  # Reset predictions
            true_labels = []  # Reset true labels
        steps+=1
    
    # Calculate the average loss over the training data
    avg_train_loss = total_loss / len(train_dataloader)


print("Training complete")

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import pandas as pd
import json

# Load the arguments data (assuming it has columns 'id', 'premise')
arguments_df = pd.read_csv('/kaggle/input/projectvalues/Project_Data/arguments-validation.tsv', delimiter='\t')

# Load the value labels data (assuming it has columns 'id', 'value_label')
labels_df = pd.read_csv('/kaggle/input/projectvalues/Project_Data/labels-validation.tsv', delimiter='\t')

# Load the value descriptions from a JSON file
with open('/kaggle/input/value-categories/value-categories (1).json', 'r') as file:
    value_descriptions = json.load(file)
    
labels_long_df = labels_df.melt(id_vars='Argument ID', var_name='value_category', value_name='label')
labels_long_df = labels_long_df[labels_long_df['value_category'] != 'Universalism: objectivity']
labels_long_df['value_description'] = labels_long_df['value_category'].apply(lambda x: value_descriptions[x.lower().replace(": ","-")]['personal-motivation'])
combined_df_val = pd.merge(arguments_df, labels_long_df, left_on='Argument ID', right_on='Argument ID')
combined_df_val['Argument'] = combined_df_val.apply(
    lambda row: f"{row['Stance']} {row['Conclusion']} by saying {row['Premise']}",
    axis=1
)

In [ ]:
test_inputs = tokenizer(list(combined_df_val['Argument']), list(combined_df_val['value_description']), padding=True, truncation=True, return_tensors="pt")
test_input_ids = test_inputs['input_ids'].to(device)
test_attention_mask = test_inputs['attention_mask'].to(device)
test_labels = torch.tensor(combined_df_val['label'].values).to(device)

test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=16)

# Function to evaluate the model on the test set
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            b_input_ids, b_attention_mask, b_labels = [b.to(device) for b in batch]

            outputs = model(b_input_ids, attention_mask=b_attention_mask)
            logits = outputs.logits

            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            batch_predictions = np.argmax(logits, axis=1)
            predictions.extend(batch_predictions)
            true_labels.extend(label_ids)

    return predictions, true_labels

# Evaluate the model
# predictions, true_labels = evaluate_model(model, test_dataloader)

# # Calculate accuracy and F1 score
# accuracy = accuracy_score(true_labels, predictions)
# f1 = f1_score(true_labels, predictions, average='binary')

# print(f"Test Accuracy: {accuracy:.4f}")
# print(f"Test F1 Score: {f1:.4f}")

In [ ]:
from torch.utils.data import Dataset,DataLoader
print(1)
class CategoryDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        premise = item['Argument']
        description = item['value_description']
        label = item['label']
        category = item['value_category']
        
        # Tokenize the text pair
        encoding = self.tokenizer(premise, description, add_special_tokens=True, 
                                  max_length=self.max_len, padding='max_length', 
                                  truncation=True, return_tensors="pt")
        
        input_ids = encoding['input_ids'].squeeze(0)  # Remove the batch dimension
        attention_mask = encoding['attention_mask'].squeeze(0)
        
        return input_ids, attention_mask, torch.tensor(label, dtype=torch.float), category
    
dataset = CategoryDataset(combined_df_val,tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    all_logits = []
    all_labels = []
    all_categories = []

    with torch.no_grad():
        for input_ids, attention_mask, labels, categories in dataloader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # Using sigmoid for binary classification
            logits = logits.cpu().numpy()
            labels = labels.cpu().numpy()
            all_logits.extend(np.argmax(logits, axis=1))
            all_labels.extend(labels)
            all_categories.extend(categories)


    # Convert lists to numpy arrays
    all_logits = np.array(all_logits)
    all_labels = np.array(all_labels)
    all_categories = np.array(all_categories)

    return all_logits, all_labels, all_categories

# Get the predictions, true labels, and categories from the evaluation
logits, labels, categories = evaluate_model(model, dataloader)


In [ ]:
f1_scores = {}
unique_categories = np.unique(categories)

for category in unique_categories:
    category_mask = (categories == category)
    cat_labels = labels[category_mask]
    cat_logits = logits[category_mask]
#     print(cat_labels.shape)
#     print(cat_logits.shape)
    # Ensure that the shape of cat_labels and cat_logits is one-dimensional
    cat_labels = np.squeeze(cat_labels)
    cat_logits = np.squeeze(cat_logits)

    # Calculate F1 score for the category
    f1 = f1_score(cat_labels, cat_logits, average='binary')
    f1_scores[category] = f1

# Print F1 scores for each category
avg_score=0
for category, score in f1_scores.items():
    print(f"F1 Score for {category}: {score:.4f}")
    avg_score+=score
avg_score=avg_score/19
print(avg_score)

In [ ]:
import pandas as pd
import json

# Load the arguments data (assuming it has columns 'id', 'premise')
arguments_df = pd.read_csv('/kaggle/input/projectvalues/Project_Data/arguments-test.tsv', delimiter='\t')

# Load the value labels data (assuming it has columns 'id', 'value_label')
labels_df = pd.read_csv('/kaggle/input/projectvalues/Project_Data/labels-test.tsv', delimiter='\t')

# Load the value descriptions from a JSON file
with open('/kaggle/input/value-categories/value-categories (1).json', 'r') as file:
    value_descriptions = json.load(file)
    
labels_long_df = labels_df.melt(id_vars='Argument ID', var_name='value_category', value_name='label')
labels_long_df = labels_long_df[labels_long_df['value_category'] != 'Universalism: objectivity']
labels_long_df['value_description'] = labels_long_df['value_category'].apply(lambda x: value_descriptions[x.lower().replace(": ","-")]['personal-motivation'])
combined_df_test = pd.merge(arguments_df, labels_long_df, left_on='Argument ID', right_on='Argument ID')
combined_df_test['Argument'] = combined_df_test.apply(
    lambda row: f"{row['Stance']} {row['Conclusion']} by saying {row['Premise']}",
    axis=1
)

In [ ]:
test_inputs = tokenizer(list(combined_df_test['Argument']), list(combined_df_test['value_description']), padding=True, truncation=True, return_tensors="pt")
test_input_ids = test_inputs['input_ids'].to(device)
test_attention_mask = test_inputs['attention_mask'].to(device)
test_labels = torch.tensor(combined_df_test['label'].values).to(device)

test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=16)

# Function to evaluate the model on the test set
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            b_input_ids, b_attention_mask, b_labels = [b.to(device) for b in batch]

            outputs = model(b_input_ids, attention_mask=b_attention_mask)
            logits = outputs.logits

            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            batch_predictions = np.argmax(logits, axis=1)
            predictions.extend(batch_predictions)
            true_labels.extend(label_ids)

    return predictions, true_labels

# Evaluate the model
# predictions, true_labels = evaluate_model(model, test_dataloader)

# # Calculate accuracy and F1 score
# accuracy = accuracy_score(true_labels, predictions)
# f1 = f1_score(true_labels, predictions, average='binary')

# print(f"Test Accuracy: {accuracy:.4f}")
# print(f"Test F1 Score: {f1:.4f}")

In [ ]:
from torch.utils.data import Dataset,DataLoader

class CategoryDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        premise = item['Argument']
        description = item['value_description']
        label = item['label']
        category = item['value_category']
        
        # Tokenize the text pair
        encoding = self.tokenizer(premise, description, add_special_tokens=True, 
                                  max_length=self.max_len, padding='max_length', 
                                  truncation=True, return_tensors="pt")
        
        input_ids = encoding['input_ids'].squeeze(0)  # Remove the batch dimension
        attention_mask = encoding['attention_mask'].squeeze(0)
        
        return input_ids, attention_mask, torch.tensor(label, dtype=torch.float), category
    
dataset = CategoryDataset(combined_df_test,tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    all_logits = []
    all_labels = []
    all_categories = []

    with torch.no_grad():
        for input_ids, attention_mask, labels, categories in dataloader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # Using sigmoid for binary classification
            logits = logits.cpu().numpy()
            labels = labels.cpu().numpy()
            all_logits.extend(np.argmax(logits, axis=1))
            all_labels.extend(labels)
            all_categories.extend(categories)


    # Convert lists to numpy arrays
    all_logits = np.array(all_logits)
    all_labels = np.array(all_labels)
    all_categories = np.array(all_categories)

    return all_logits, all_labels, all_categories

# Get the predictions, true labels, and categories from the evaluation
logits, labels, categories = evaluate_model(model, dataloader)


In [ ]:
f1_scores = {}
unique_categories = np.unique(categories)

for category in unique_categories:
    category_mask = (categories == category)
    cat_labels = labels[category_mask]
    cat_logits = logits[category_mask]
#     print(cat_labels.shape)
#     print(cat_logits.shape)
    # Ensure that the shape of cat_labels and cat_logits is one-dimensional
    cat_labels = np.squeeze(cat_labels)
    cat_logits = np.squeeze(cat_logits)

    # Calculate F1 score for the category
    f1 = f1_score(cat_labels, cat_logits, average='binary')
    f1_scores[category] = f1

# Print F1 scores for each category
avg_score=0
for category, score in f1_scores.items():
    print(f"F1 Score for {category}: {score:.4f}")
    avg_score+=score
avg_score=avg_score/19
print(avg_score)

In [ ]:
torch.save(model, 'model_entailment.pth')